# DeepDream

In [1]:
from keras.applications import inception_v3
from keras import backend as K

In [27]:
K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [4]:
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 0.5,
}

In [5]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
layer_dict

{'activation_1': <keras.layers.core.Activation at 0x1b436a0ce10>,
 'activation_10': <keras.layers.core.Activation at 0x1b4380b84e0>,
 'activation_11': <keras.layers.core.Activation at 0x1b438169e48>,
 'activation_12': <keras.layers.core.Activation at 0x1b438272860>,
 'activation_13': <keras.layers.core.Activation at 0x1b438326240>,
 'activation_14': <keras.layers.core.Activation at 0x1b43834cac8>,
 'activation_15': <keras.layers.core.Activation at 0x1b43842e438>,
 'activation_16': <keras.layers.core.Activation at 0x1b4384d1e10>,
 'activation_17': <keras.layers.core.Activation at 0x1b43858df98>,
 'activation_18': <keras.layers.core.Activation at 0x1b438642390>,
 'activation_19': <keras.layers.core.Activation at 0x1b43873b860>,
 'activation_2': <keras.layers.core.Activation at 0x1b437baceb8>,
 'activation_20': <keras.layers.core.Activation at 0x1b438799320>,
 'activation_21': <keras.layers.core.Activation at 0x1b4388a6dd8>,
 'activation_22': <keras.layers.core.Activation at 0x1b4389097b8

In [12]:
loss = K.variable(0.)
loss

<tf.Variable 'Variable_2:0' shape=() dtype=float32_ref>

In [13]:
# keras.backend.prod(x, axis=None, keepdims=False)
# 在某一指定轴，计算张量中的值的乘积。
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling
#WARNING:tensorflow:Variable += will be deprecated. 
#Use variable.assign_add if you want assignment to the variable value or 'x = x + y' if you want a new python Tensor object.

In [16]:
dream = model.input

grads = K.gradients(loss, dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

In [18]:
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

In [19]:
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [21]:
import scipy
import numpy as np
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1
              )
    return scipy.ndimage.zoom(img, factors, order=1)

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)
    
def preprocess_image(img_path):
    img = image.load_img(img_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

In [26]:
step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.

base_image_path = 'creative_commons_elephant.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    #save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')

Processing image shape (179, 267)
...Loss value at 0 : 1.503927
...Loss value at 1 : 1.8687117
...Loss value at 2 : 2.506299
...Loss value at 3 : 3.1465912
...Loss value at 4 : 3.7639017
...Loss value at 5 : 4.354712
...Loss value at 6 : 4.8810725
...Loss value at 7 : 5.4355063
...Loss value at 8 : 5.9943447
...Loss value at 9 : 6.409588
...Loss value at 10 : 6.929902
...Loss value at 11 : 7.417097
...Loss value at 12 : 7.9169416
...Loss value at 13 : 8.38111
...Loss value at 14 : 8.768145
...Loss value at 15 : 9.226697
...Loss value at 16 : 9.546183
Processing image shape (250, 375)
...Loss value at 0 : 2.642964
...Loss value at 1 : 3.9256825
...Loss value at 2 : 5.021464
...Loss value at 3 : 5.931832
...Loss value at 4 : 6.7895675
...Loss value at 5 : 7.6924334
...Loss value at 6 : 8.539401
...Loss value at 7 : 9.598038
Processing image shape (351, 525)
...Loss value at 0 : 3.313401
...Loss value at 1 : 5.69628
...Loss value at 2 : 8.687907


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
